In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd /content/drive/MyDrive/Georgia Tech/Deep Learning/Project/common-workspace

In [ ]:
!pip install git+https://github.com/facebookresearch/esm.git
# !pip install -e esm
!pip install -e .

In [ ]:
import torch
import esm
import math
import time

from pgen import sampler_1
from pgen import vanilla_esm34
from pgen import vanilla_esm6

In [ ]:
model = vanilla_esm34.ESM34()

In [ ]:
sampler = sampler_1.Sampler_1(model,device="gpu")

In [ ]:
ref = "MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"
indexes = [39, 40, 41, 54]
n_samples = 300

#Mimic old sequential sampler
#TODO: maybe patch the sampler to mask the positions of interest until something is generated for them?
sequential_results = sampler.generate(n_samples, seed_seq=ref, batch_size=10, in_order=True, max_len=len(ref), indexes=indexes, num_positions=1, num_iters=len(indexes), mask=True)

#Generate all positions in parallel
parallel_results = sampler.generate(n_samples, seed_seq=ref, batch_size=10, in_order=True, max_len=len(ref), indexes=indexes, num_positions=len(indexes), num_iters=1, mask=True)

#Generate all positions in parallel, but only take the most likely sequence
parallel_k_1 = sampler.generate(1, seed_seq=ref, batch_size=1, top_k=1, in_order=True, max_len=len(ref), indexes=indexes, num_positions=len(indexes), num_iters=1, mask=True)



In [ ]:
# Load in fitness data
import csv
fitness_data = {}
fitness_data_list = []
with open('data/elife-16965-supp1.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader) # Skip header
  for row in reader:
    fitness_data[row[0]] = float(row[4])
    fitness_data_list.append(float(row[4]))

In [ ]:
# Extract mutations from results
scores = []
for result in results:
  mutation = ""
  for i in indexes:
    mutation += result[i-1]
  scores.append(fitness_data.get(mutation, 0))

In [ ]:
# Generate random scores
import random
rand_scores = []
max_size = len(fitness_data)
for i in range(n_samples):
  idx = random.randrange(0, max_size)
  rand_scores.append(fitness_data_list[idx])

In [ ]:
print('min', min(scores))
print('max', max(scores))
print('mean', sum(scores) / n_samples)

In [ ]:
print('min', min(rand_scores))
print('max', max(rand_scores))
print('mean', sum(rand_scores) / n_samples)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

random_data = np.array(rand_scores)
fig1, ax1 = plt.subplots()
ax1.set_title('Random')
ax1.boxplot(data)

esm_data = np.array(scores)
fig2, ax2 = plt.subplots()
ax2.set_title('ESM')
ax2.boxplot(esm_data)